# TensorFlow(Keras) のコードを SageMaker Training 向けに適したコードに書き換える
* [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) の画像分類問題を解く

## 書き換え前のベースのコード
そのまま動くことを確認します。

In [ ]:
import tensorflow as tf
from keras import backend as K
K.set_image_data_format('channels_last')

# データロード
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
x_train = (x_train/255)
x_valid = (x_valid/255)

# モデル定義
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))

# トレーニング
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['accuracy'],loss="sparse_categorical_crossentropy")
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model.save('./1')

## 書き換え前のコードが SageMaker Training で動くことも確認
処理自体は動くけれども出来上がったモデルも揮発する

In [ ]:
!pygmentize ./src/2-1-1/train.py

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/2-1-1/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit()

## モデルを S3 に保存するように書き換え  

### (coding) トレーニングスクリプトの書き換え
以下のセルを実行(shift+enter)することで、`%%writefile` というマジックコマンドによって `%%writefile` の後に指定した場所にセルの内容が保存されます。モデルを S3 に保存するように書き換えて実行して、トレーニングジョブを動かしましょう。  
書き換え場所や書き換え内容がわからない方はヒントを 4 つ用意しましたので参照してください。(ヒントと書かれている部分をクリックするとヒントの内容が見えるようになります）  


<details><summary>▶ヒント1: 書き換え場所</summary><div>

モデルを指定ディレクトリに保存するとトレーニング完了時に Amazon S3 に自動で転送されます。`モデルの保存`をしている場所を書き換える必要があります。

</div></details>

<details><summary>▶ヒント2: モデルの保存先</summary><div>

トレーニング完了時に Amazon S3 に自動で転送するには環境変数で定義された指定ディレクトリに保存する必要があります。

</div></details>

<details><summary>▶ヒント3: 使用する環境変数</summary><div>

`SM_MODEL_DIR` と `SM_OUTPUT_DATA_DIR` で指定された値のディレクトリに保存するとトレーニング完了時に Amazon S3 に自動で転送します。今回のケースでより適しているのは？（どちらに保存しても問題なく動作はします）

</div></details>

<details><summary>▶ヒント4: 環境変数を Python のコードで取得するには？</summary><div>

`os` モジュールの `os.environ.get('{環境変数名}')` もしくは `os.getenv()` を使用するのが一般的です。  
`getenv` は `envrion.get`同じ（エイリアス）なので、どちらを使用しても良いです。  
 事前に `import os` する必要があることに注意してください  

</div></details>

<details><summary>▶ヒント5: ディレクトリとファイル名をつなげる良い方法</summary><div>

ディレクトリは末尾がスラッシュがつくのかつかないのかわからないケースがあり、パスの文字列を作るとき混乱します。  
例：`/opt/ml/code` なのか `/opt/ml/code/` なのか、そこに例えば `train.py` を指定するとき、単純に文字列連結してしまうと、  
`/opt/ml/codetrain.py`となってしまったり、`/opt/ml/code//train.py`となってしまったりしまいます。  
`os` モジュールの `os.path.join(DIRECTORY,FILE)`を使うと、スラッシュの有無を気にせずにパスをきれいに生成してくれます。

</div></details>

In [ ]:
%%writefile ./src/2-1-2/train.py
import tensorflow as tf
from keras import backend as K
K.set_image_data_format('channels_last')

# データロード
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
x_train = (x_train/255)
x_valid = (x_valid/255)

# モデル定義
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))

# トレーニング
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['accuracy'],loss="sparse_categorical_crossentropy")
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model.save('./1')

### トレーニングジョブの実行
上記で作成したトレーニングスクリプトを実行してみます。  
SageMaker Notebook を利用している場合は `instance_type='local'` とするとトレーニングジョブを Notebook インスタンスで実行できるため起動が早く試しやすいです。

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/2-1-2/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit()

### モデルの確認
モデルが S3 に転送されているかを確認するため、S3 からモデルをダウンロードしてロードしてみます。

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()

## トレーニングデータを S3 からトレーニングインスタンスに転送する

トレーニングジョブでアーティファクト（今回ではモデル）を S3 に連携できました。次はトレーニングデータをトレーニングスクリプトを実行前に Amazon S3 から連携させます。
### データを Amazon S3 に送り込む
以下コードを実行して、Amazon S3 にトレーニングデータを Amazon S3 にデータを送り込みます。  
データの形式は元が ndarray のため、そのまま npy 形式で保存することとします。  
送り込んだ先の URI は `train_s3_uri`, `valid_s3_uri` に格納されます。

In [ ]:
!mkdir -p ./train
!mkdir -p ./valid

import tensorflow as tf
import numpy as np
import sagemaker

(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_valid = x_valid/255

np.save('./train/x_train.npy',x_train)
np.save('./train/y_train.npy',y_train)
np.save('./valid/x_valid.npy',x_valid)
np.save('./valid/y_valid.npy',y_valid)
train_s3_uri = sagemaker.session.Session().upload_data(path='./train/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/2-1/train')
valid_s3_uri = sagemaker.session.Session().upload_data(path='./valid/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/2-1/valid')

### (coding) トレーニングコードの書き換え
`fit` メソッド実行時に引数に以下の引数を指定してトレーニングインスタンスにトレーニングデータを送り込みます。
`fit({'train':train_s3_uri,'valid':valid_s3_uri})`  
トレーニングインスタンスに送り込まれたデータを使ってトレーニングするようにトレーニングコードを書き換えましょう。


<details><summary>▶ヒント1: 書き換え場所</summary><div>

今までは TensorFlow のモジュールにある `load_data()` メソッドでデータを読み込んでいました。  
`load_data()` 部分をファイルからロードするように書き換える必要があります。

</div></details>

<details><summary>▶ヒント2: ロードするファイルの場所</summary><div>

トレーニングスクリプトを実行する時のカレントディレクトリは `/opt/ml/code` であり、トレーニングスクリプトが配置された場所です。  
そこに `.npy` ファイルは存在しないため、パスを指定してファイルを読み込む必要があります。  
ファイルの場所は環境変数で定義された `SM_CHANNEL_{辞書形式で渡したfitの引数のキー名の大文字}` の値のディレクトリです。

</div></details>

<details><summary>▶ヒント3: `.npy` ファイルの読み込み方</summary><div>

`np.load(FILE_PATH)` で読み込みます。

</div></details>


In [ ]:
%%writefile ./src/2-1-3/train.py
import tensorflow as tf
from keras import backend as K
K.set_image_data_format('channels_last')
import os

# データロード
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_valid = x_valid/255

# モデル定義
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['accuracy'],loss="sparse_categorical_crossentropy")

# トレーニング
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model_dir = os.environ.get('SM_MODEL_DIR')
model.save(os.path.join(model_dir,'1'))

### トレーニングジョブの実行

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/2-1-3/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

### モデルの確認

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!rm model.tar.gz
!rm -rf ./1
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()

## ハイパーパラメータをトレーニングスクリプト外から入力する

### ハイパーパラメータを Estimator で定義する
`batch_size` や `epochs` などのパラメータはいろいろ試して良い数値を探索することがあります。  
毎回トレーニングコードを書き換えるのではなく外部からパラメータを設定できたほうが何かと便利です。  
SageMaker では Estimator インスタンス生成時の引数で `hyperparameters` という引数を用いることでトレーニングスクリプトの外からパラメータを与えることができます。  
ここでは以下のハイパーパラメータを与えたときにそのように動くようにトレーニングスクリプトを書き換えましょう。

```python
hyperparameters={
    'filters': 8,
    'epochs': 1,
    'batch_size': 16,
    'learning_rate': 0.001
}
```
<details><summary>▶ヒント1: スクリプト実行時にどうやってハイパーパラメータがスクリプトに引き渡されるか</summary><div>

3 つあり、どれを使ってもよいです。それぞれ default 値を設定しておくと良いでしょう。

1. コマンドライン引数
  トレーニングスクリプトを実行する際に以下のように実行されます。
  `python train.py --filters 8 --epochs 1 --batch_size 16 --learning_rate 0.001 --model_dir s3://{bucket}/prefix`
  [argparse モジュール](https://docs.python.org/ja/3/howto/argparse.html)を使って受け取ると良いでしょう。
  ユーザが設定していないハイパーパラメータ `model_dir` (モデルが S3 のどこに保存されるか)が自動で設定されることに注意しましょう。
  default 値を設定する場合は、`add_argument()` を実行する際に、default 引数を設定します。

2. SM_HPS 環境変数
  SM_HPS という環境変数に json 文字列で格納されるので、それを取得するのもよいでしょう。  
  以下のように取得すると `hps` 変数に辞書形式で取得できます。  
  
  ```python
  import json, os
  hps = json.loads(os.environ.get('SM_HPS'))
  ```
  デフォルト値は `hps.setdefault(KEY,VALUE)`を利用すると良いでしょう。
  
3. SM_HP_{ハイパーパラメータ名大文字}
  `os.environ.get()`で取得できます。デフォルト値の設定には第2引数にデフォルト値を設定します。

</div></details>


In [ ]:
%%writefile ./src/2-1-4/train.py
import tensorflow as tf
from keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np

# データロード
train_dir = os.environ.get('SM_CHANNEL_TRAIN')
valid_dir = os.environ.get('SM_CHANNEL_VALID')
x_train = np.load(os.path.join(train_dir,'x_train.npy'))
y_train = np.load(os.path.join(train_dir,'y_train.npy'))
x_valid = np.load(os.path.join(valid_dir,'x_valid.npy'))
y_valid = np.load(os.path.join(valid_dir,'y_valid.npy'))

# モデル定義
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['accuracy'],loss="sparse_categorical_crossentropy")

# トレーニング
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model_dir = os.environ.get('SM_MODEL_DIR')
model.save(os.path.join(model_dir,'1'))

### トレーニングの実行
設定したハイパーパラメータが使われているか確認しましょう。

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/2-1-4/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'filters': 8,
        'epochs': 1,
        'batch_size': 16,
        'learning_rate': 0.001
    }
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

### 出来上がったモデルの確認

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!rm model.tar.gz
!rm -rf ./1
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()